In [164]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
import re
import string

# Tides processing

In [223]:
root_path = os.getcwd()
tides_root_path = os.path.join(root_path, 'tides')
tides_data_path = os.path.join(tides_root_path, 'raw')

In [224]:
tides_root_path

'E:\\#PROJECT\\floodmap\\tides'

In [225]:
tides_data_path

'E:\\#PROJECT\\floodmap\\tides\\raw'

In [23]:
tides_data_list = os.listdir(tides_data_path)

In [24]:
len(tides_data_list)

8210

In [34]:
# Coordinate processor from filename
np.array(tides_data_list[0].split('][')[2].split(',')).astype('float32')

array([ 95.01131 , -10.987473], dtype=float32)

In [100]:
# Extract tides value
extracted_tides = []
coordinates = []
flag_duplicate_coordinate = 0
flag_not_enough_data = 0 
flag_invalid = 0

for i, tides_data in enumerate(tides_data_list):
    tides_file_path = os.path.join(tides_data_path, tides_data)
    # tides_file_path = os.path.join(tides_data_path, '[123][30][138.54195170940173,0.29589898989899055][hasil(2)-1602398717703].txt')
    with open(tides_file_path, 'r') as f:
        data = f.readlines()
        
    coordinate = tides_data.split('][')[2].split(',')
    
    # Check if same coordinate have been processed before
    if coordinate in coordinates:
        flag_duplicate_coordinate += 1
        # print(f'{i}/{len(tides_data_list)} duplicate coordinate')
        continue

    # value = np.array([d.strip()[-5:] for d in data[3:]]).astype('float32')
    # coordinate = np.array(tides_data_list[0].split('][')[2].split(',')).astype('float32')
    
    # Valid tides data:
    # 1. Index 0,1,2 are header
    # 2. Data start from index 3
    # 3. Have at least 360 data (if more are available, trimmed.)
    if len(data) < 363:
        flag_not_enough_data += 1
        # print(f'{i}/{len(tides_data_list)} not enough data')
        continue
    
    value = [d.strip()[-5:] for d in data[3:363]]
    
    # Check if extracted value is valid one, by detecting
    # `*` pattern
    if re.search('\*', value[0]):
        flag_invalid += 1
        # print(f'{i}/{len(tides_data_list)} isn\'t valid')
        continue
    else:
        pass
    
    
    extracted_tides.append([coordinate,value])
    coordinates.append(coordinate)
    if i % 100 == 99:
        print(f'{i}/{len(tides_data_list)}')

# Store to JSON
extracted_tides_filename = 'extracted.json'
extracted_tides_path = os.path.join(tides_root_path, extracted_tides_filename)
with open(extracted_tides_path, 'w') as f:
    json.dump(extracted_tides, f)
    

4699/8210


In [101]:
flag_duplicate_coordinate, flag_not_enough_data, flag_invalid

(3165, 109, 1748)

# Run excel VBA, manipulate cell value, read back cell value

In [228]:
# Write input value
wb = ox.load_workbook(filename=tides_engine_path, read_only=False, keep_vba=True)
inputs = wb['Input']

with open(extracted_tides_path, 'r') as f:
    extracted_tides = json.load(f)

start_row = 9
column_ranges = string.ascii_uppercase[2:]

for i, value in enumerate(extracted_tides[3000][1]):
    if i % (len(column_ranges)) == 0:
        start_row += 1
    
    column = i % len(column_ranges)
    
    cell = f'{column_ranges[column]}{start_row}'
    inputs[cell] = float(value)

wb.save(tides_excel_path)
wb.close()

In [226]:
import win32com.client

tides_engine_path = os.path.join(tides_root_path, 'tides_engine.xlsm')
# tides_excel_path = 'J:\Kuliah\Mata Kuliah\Proteksi Pantai dan Rawa\PrediksiPasut_Teluk Palu - Copy\PrediksiPasut.xlsm'

xl = win32com.client.Dispatch("Excel.Application")

wb = xl.Workbooks.Open(tides_engine_path)
xl.Application.Run('tides_engine.xlsm!Sheet4.cmdRefresh_Click()')

wb.Save()
xl.Application.Quit()

In [229]:
import openpyxl as ox

# Read existing workbook
wb = ox.load_workbook(filename=tides_engine_path, read_only=False, keep_vba=True)
results = wb['Calculation']

Z0 = results['K4'].value
M2 = results['K5'].value
S2 = results['K6'].value
N2 = results['K7'].value
K2 = results['K8'].value
K1 = results['K9'].value
O1 = results['K10'].value
P1 = results['K11'].value
M4 = results['K12'].value
MS4 = results['K13'].value

HHWL = Z0 + S2 + K1 + O1 + P1 + M2 + K2
LLWL = Z0 - (M2 + S2 + N2 + K1 + O1 + P1 + M4 + MS4)
MHWL = Z0 + (M2 + S2)
MLWL = Z0 - (M2 + S2)

print(HHWL, MHWL, Z0, MLWL, LLWL)

wb.close()

0.8029214984271281 0.40161706773386086 0.21142364514928993 0.021230222564719042 -0.37316177895413327


1: 0.5103402630290366 0.24000239396078965 0.21253740617953432 0.18507241839827898 -0.2396688722988668
100: 0.9433209812519656 0.5183703025896438 0.35372692542859147 0.18908354826753915 -0.4143937489349982
500: 0.5520498851993269 0.2561199412571001 0.2018665634657847 0.1476131856744693 -0.2670258279947593
1000: 0.589483074951265 0.2822054405206289 0.20717580564938662 0.1321461707781443 -0.27599377855357965
2000: 0.4674378889654878 0.2088560435286184 0.17832787654229132 0.14779970955596425 -0.23469777330249147
3000: 0.8029214984271281 0.40161706773386086 0.21142364514928993 0.021230222564719042 -0.37316177895413327

360

In [189]:
import openpyxl as ox

# Read existing workbook
wb = ox.load_workbook(filename=tides_engine_path, read_only=False, keep_vba=True)
results = wb['Calculation']

Z0 = results['K4'].value
M2 = results['K5'].value
S2 = results['K6'].value
N2 = results['K7'].value
K2 = results['K8'].value
K1 = results['K9'].value
O1 = results['K10'].value
P1 = results['K11'].value
M4 = results['K12'].value
MS4 = results['K13'].value

HHWL = Z0 + S2 + K1 + O1 + P1 + M2 + K2
LLWL = Z0 - (M2 + S2 + N2 + K1 + O1 + P1 + M4 + MS4)
MHWL = Z0 + (M2 + S2)
MLWL = Z0 - (M2 + S2)

print(HHWL, MHWL, Z0, MLWL, LLWL)

wb.close()

1.4184608264202618 0.9080172405619822 -0.0003460574947640893 -0.9087093555515104 -1.432460338137139
